In [2]:
import pandas as pd
import SDSSdownload as sd
import glob

# astropy will yell about deprecated FITS stuff, as usual
import warnings
warnings.filterwarnings("ignore")

import os
if not os.path.exists('downloaded_galaxies.txt'):
    with open('downloaded_galaxies.txt', "x") as txt:
        txt.write("#JID,objID,on_plate\n")

In [3]:
cat = pd.read_csv("NairAbrahamMorphology.cat", delim_whitespace=True)
sdss_cat_path = 'SDSS_query.csv'
sdss_cat = pd.read_csv(sdss_cat_path)

In [5]:
a = 0 # starting index
b = len(glob.glob('./data/*/')) # number of galaxies downloaded
c = 1000 # number to download
start_index = a+b
end_index = a+b+c
print(a+b,a+b+c)

1000 2000


In [ ]:
new_cat = cat.iloc[start_index:end_index]

# For this each of these, we'll download just the i, r, and g filters
bands = ['i','r','g']

for JID, ra, dec, p90, z in zip(new_cat['#JID'], new_cat.RA, new_cat.DEC, new_cat.Rp90_g, new_cat.z):
    # Download and make cutouts
    dl = sd.Image(ra=ra, dec=dec, bands=bands, df_path=sdss_cat_path)
    size = sd.size_from_z(p90, z)
    dl.cutout(size=size)

    # Make B&W images for each band
    for band in bands:
        dl.save_plot(band=band)

    # Make a color image
    dl.save_cutout_color(red='i', green='r', blue='g')

    # Append text file with values
    with open('downloaded_galaxies.txt', 'a') as txt:
        txt.write(f"{JID}, {dl.objid}, {dl.on_plate}\n")